<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Global Neural Network Model
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook visualises the main results from the paper.
</p>

</div>


##  1. Loading Packages and Data

In this section, we import required libraries, define the model parameters and load the dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
from models.global_model.model_functions.helper_functions.prepare_data import Prepare
from utils import create_pred_input
from scipy.interpolate import griddata
from models import MultivariateModelGlobal as Model       


os.environ['PYTHONHASHSEED'] = str(0)

#model parameters                    
lr = 0.001                      # Learning rate
min_delta = 1e-6               # Tolerance for optimization
patience = 100                   # Patience for early stopping
verbose = 2                     # Verbosity mode for optimization
n_countries=196
time_periods=63                 #

#prepare the data
data=pd.read_excel('../data/MainData.xlsx')
growth, precip, temp = Prepare(data, n_countries, time_periods)
x_train = {0:temp, 1:precip}

#summary statics for standardisation
mean_temp=np.nanmean(data["TempPopWeight"])
std_temp=np.nanstd(data["TempPopWeight"])
mean_precip=np.nanmean(data["PrecipPopWeight"])
std_precip=np.nanstd(data["PrecipPopWeight"])

pred_input, T, P= create_pred_input(mc=False, mean_T=mean_temp, std_T=std_temp, mean_P=mean_precip, std_P=std_precip)




2025-09-03 08:25:54.987986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 08:25:55.085276: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 08:25:55.170181: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 08:25:55.266844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756887955.347512   16068 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756887955.35

In [ ]:
#load results from simulations and retrieve best n models
n_models = 5  
date_of_run = '2025-08-26'
Model_selection='IC'

results = dict(np.load(f'../results/metrics/{Model_selection}/{date_of_run}/results.npy', 
                       allow_pickle=True).item())

results={k: v for k,v in results.items() if v is not None}
top_models = sorted(results, key=lambda node: results[node][1])[0:n_models]

#print the corresponding BIC for all  the top models

print(f'Top models {top_models}, BIC: {[results[model][1] for model in top_models]}')



Top models [(4,), (2, 2), (8,), (4, 2), (4, 4)], BIC: [np.float64(-57499.217679141926), np.float64(-57496.46984861002), np.float64(-57482.26543484042), np.float64(-57475.700707413875), np.float64(-57473.425462418665)]


# 2. Data analysis

## 2.1 Calculating the best ten surfaces, the average of those and the benchmark surface 


In [ ]:
Model_selection='BIC'
date_of_run = '2025-08-29'
# --- Build surfaces for each of the top-n models ---------------------
model_surfaces = []
for idx, node in enumerate(top_models[0:n_models], 1):
    
    print(node)
    # instantiate and load your model
    factory = Model(node, x_train, growth, dropout=0.2, penalty=0, country_trends=False, dynamic_model=True)
    factory.Depth=len(node)
    model=factory.get_model()
    weight_file = f'../results/Model Parameters/{Model_selection}/{date_of_run}/{node}.weights.h5'
    model.load_params(weight_file)

    # fit & predict
    model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)
   
    pred_flat = model.model_visual.predict(pred_input).reshape(-1,)
    Growth = pred_flat.reshape(T.shape) 

    opacity = 0.3
    surf = go.Surface(
        x=T, y=P/1000, z=Growth, #ensure that the surfaces are meassured in meters instead of milimeters
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name=f'Model {node}'
    )
    model_surfaces.append(surf)
 

#calculate the average surface
z=np.mean([surf.z for surf in model_surfaces], axis=0).reshape(T.shape)

mean_surface = go.Surface(
        x=T, y=P, z=z,
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name='mean_surface'
    )


# --- Load benchmark data and create grid  ---------------------

benchmark_data = pd.read_csv('../data/Benchmark/3d_results.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values

bench_Z = griddata(
    points=(bench_temp, bench_precip),
    values=bench_growth,
    xi=(T, P),
    method='linear'
)

# --- Create the benchmark surface --------------------------------------
bench_surface = go.Surface(
    x=T, y=P, z=bench_Z,
    colorscale='Reds',
    showscale=False,
    name='Benchmark'
)





(2, 2)


ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense_40, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(195, 1), Received: value.shape=(62, 1). Target variable: <Variable path=dense_40/kernel, shape=(195, 1), dtype=float32, value=[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

### 2.1.2 Data visualisation
In this section we visualise the top-n models in a temperature-precipitation grid 


In [ ]:

plot_data = model_surfaces
# # plot_data = [mean_surface] 
# plot_data=[mean_surface, bench_surface]


fig1 = go.Figure(data=plot_data)
fig1.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',
        camera=dict(eye=dict(x=2.11, y=0.12, z=0.38)),
        zaxis=dict(range=[-0.15, 0.15])
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig1.show()








In [ ]:

fig.write_image("../results/images/Paper/3d_plot_model_vs_Leirvik.pdf", width=1600, height=1200, scale=2)



In [ ]:
# --- optionally: Save the figure as HTML ---------------------------------------------
fig.write_html(f'../results/images/10_best_average_vs_Leirvik.html')

## In this section I compare the in-sample fit of our Neural Network and a quadratic benchmark

In [5]:
node=(2,)    
# instantiate and load your model
factory = Model(node, x_train, growth, dropout=0.2, penalty=0, country_trends=False)
factory.Depth=len(node)
model=factory.get_model()
weight_file = f'../results/Model Parameters/BIC/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

pred_flat = model.model_visual.predict(pred_input).reshape(-1,)
Growth = pred_flat.reshape(T.shape) 

opacity = 0.3
surf = go.Surface(
    x=T, y=P/1000, z=Growth, #ensure that the surfaces are meassured in meters instead of milimeters
    colorscale='Cividis',
    opacity=0.85,
    showscale=False,
    name=f'Model {node}'
)

fig1 = go.Figure(data=surf)
fig1.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',
        camera=dict(eye=dict(x=2.11, y=0.12, z=0.38)),
        zaxis=dict(range=[-0.15, 0.15])
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig1.show()




2025-08-26 13:25:53.803867: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/1000000


2025-08-26 13:26:01.079931: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


1/1 - 5s - 5s/step - loss: 0.0036
Epoch 2/1000000
1/1 - 0s - 133ms/step - loss: 0.0036
Epoch 3/1000000
1/1 - 0s - 91ms/step - loss: 0.0036
Epoch 4/1000000
1/1 - 0s - 126ms/step - loss: 0.0036
Epoch 5/1000000
1/1 - 0s - 118ms/step - loss: 0.0036
Epoch 6/1000000
1/1 - 0s - 99ms/step - loss: 0.0036
Epoch 7/1000000
1/1 - 0s - 95ms/step - loss: 0.0036
Epoch 8/1000000
1/1 - 0s - 91ms/step - loss: 0.0036
Epoch 9/1000000
1/1 - 0s - 103ms/step - loss: 0.0036
Epoch 10/1000000
1/1 - 0s - 93ms/step - loss: 0.0036
Epoch 11/1000000
1/1 - 0s - 120ms/step - loss: 0.0036
Epoch 12/1000000
1/1 - 0s - 141ms/step - loss: 0.0036
Epoch 13/1000000
1/1 - 0s - 140ms/step - loss: 0.0036
Epoch 14/1000000
1/1 - 0s - 112ms/step - loss: 0.0036
Epoch 15/1000000
1/1 - 0s - 127ms/step - loss: 0.0036
Epoch 16/1000000
1/1 - 0s - 131ms/step - loss: 0.0036
Epoch 17/1000000
1/1 - 0s - 105ms/step - loss: 0.0036
Epoch 18/1000000
1/1 - 0s - 98ms/step - loss: 0.0036
Epoch 19/1000000
1/1 - 0s - 112ms/step - loss: 0.0036
Epoch 20

/workspaces/Paper_1/.venv/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['X_in']
Received: inputs=Tensor(shape=(1, 1, 8100, 2))
  warnings.warn(msg)


In [30]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance


class SKTFWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        return self  # model already trained

    def predict(self, X):
        # Ensure correct shape (n_samples, n_features)
        X = np.array(X)
        return self.model.predict(X).reshape(-1)

    def score(self, X, y):
        y_pred = self.predict(X)
        u = np.sum((y - y_pred) ** 2)
        v = np.sum((y - np.mean(y)) ** 2)
        return 1 - u/v



wrapped_model = SKTFWrapper(model)


perm = PermutationImportance(
    wrapped_model, 
    random_state=1, 
    scoring="r2"  # or "neg_mean_squared_error"
).fit(x_train.values, growth.values)   # convert both X and y to NumPy


eli5.show_weights(perm, feature_names=x_train.columns.tolist())

ValueError: Expected 2D array, got scalar array instead:
array=<built-in method values of dict object at 0x7f5106994f80>.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
best_model.in_sample_predictions()
print("AIC", best_model.BIC)

AIC -55524.05918969524


2.3 Marginal effects

In [ ]:
   ##Marginal effects are calculated by keeping precipitation at a constant, and then calculating the corresponding growth moving one temperature up. 

   # flatten the precip grid and compute 33rd percentile
   fixed_array = np.percentile(np.array(P.flatten()), q=99)

   # find the row index in P closest to that precip
   # (assuming P is shape (n_precip, n_temp) from meshgrid)
   idx = np.argmin(np.abs(P[:,0] - fixed_array))

   axis = T[idx, :]


   # extract model and benchmark slices
   model_slice = Growth[idx, :]
   bench_growth = bench_Z[idx, :]


   marginal_effects_NN = np.diff(model_slice)  # first difference to approximate marginal effects
   marginal_effects_Leirvik = np.diff(bench_growth)  # first difference to approximate marginal effects

   fig2d = go.Figure()



   # --- 2) make the 2D line plot --------------------------------------------
   fig2d.add_trace(go.Scatter(
            x=axis,
            y=marginal_effects_NN,
            mode='lines',
            line=dict(width=2)
      ))

   fig2d.add_trace(go.Scatter(
            x=axis,
            y=marginal_effects_Leirvik,
            mode='lines',
            line=dict(width=2)
      ))



   fig2d.update_layout(
   xaxis_title='Temperature (°C)',
   yaxis_title='Growth',
   xaxis=dict(range=[0, 30]),
   legend=dict(
      bgcolor='rgba(255,255,255,0.7)',
      bordercolor='black',
      borderwidth=1
   )
   )

   fig2d.show()





   

## 2.2 Comparing Fixed Effects

In [1]:
#model parameters                    
lr = 0.001                      # Learning rate
min_delta = 1e-6               # Tolerance for optimization
patience = 50                   # Patience for early stopping
verbose = 2                     # Verbosity mode for optimization
n_countries=196
time_periods=63                 #

#define model of interest
node=(2,)
factory = Model(node, x_train, growth, dropout=0.2, penalty=0, country_trends=True)
factory.Depth=len(node)
model=factory.get_model()

weight_file = f'../results/Model Parameters/BIC/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

model.alpha
theta_linear = model.linear_trend_layer.get_weights()[0].reshape(-1)  # length = number of countries included
theta_quad   = model.quadratic_trend_layer.get_weights()[0].reshape(-1)



NameError: name 'Model' is not defined

In [34]:

#compare country Fixed effects from benchmark and model 

bench_country_FE=np.load('../Benchmark/country_FE.npy', allow_pickle=True)


iso = np.asarray(bench_country_FE)[:,0]
arr = np.asarray(bench_country_FE)[:,1]
alpha_1d = np.asarray(model.alpha).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_country_FE': arr,
    'model_alpha': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

(195,) (195,)
               iso bench_country_FE  model_alpha      diff
0      C(ISO)[T.8]         2.724419    -0.013990  2.738409
1     C(ISO)[T.12]         2.838405    -0.006018  2.844423
2     C(ISO)[T.16]         3.203189     0.061204  3.141985
3     C(ISO)[T.24]         2.652902    -0.003291  2.656194
4     C(ISO)[T.28]         3.044317    -0.016146  3.060463
..             ...              ...          ...       ...
190  C(ISO)[T.858]         2.824695    -0.019081  2.843776
191  C(ISO)[T.860]         2.578213     0.011377  2.566836
192  C(ISO)[T.882]         2.806298     0.004505  2.801793
193  C(ISO)[T.887]         2.701435    -0.007037  2.708472
194  C(ISO)[T.894]         2.849093     0.019768  2.829325

[195 rows x 4 columns]


In [36]:
#compare country Fixed effects from benchmark and model 

bench_time_FE=np.load('../Benchmark/time_FE.npy', allow_pickle=True)



iso = np.asarray(bench_time_FE)[:,0]
arr = np.asarray(bench_time_FE)[:,1]
alpha_1d = np.asarray(model.beta).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_time_FE': arr,
    'model_alpha': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

(62,) (62,)
                iso bench_time_FE  model_alpha      diff
0   C(Year)[T.1962]      0.012038     0.014021 -0.001982
1   C(Year)[T.1963]      0.010864     0.010663  0.000202
2   C(Year)[T.1964]      0.018263     0.019452 -0.001189
3   C(Year)[T.1965]      0.017382     0.016576  0.000806
4   C(Year)[T.1966]       0.00874     0.006543  0.002197
..              ...           ...          ...       ...
57  C(Year)[T.2019]      0.000793     0.002026 -0.001233
58  C(Year)[T.2020]     -0.076118    -0.074507  -0.00161
59  C(Year)[T.2021]      0.030084     0.030975 -0.000892
60  C(Year)[T.2022]      0.014908     0.017487 -0.002579
61  C(Year)[T.2023]      0.001632     0.002877 -0.001245

[62 rows x 4 columns]


In [37]:
#compare country Fixed effects from benchmark and model 

bench_linear_trend_FE=np.load('../Benchmark/linear_time_trend.npy', allow_pickle=True)



iso = np.asarray(bench_linear_trend_FE)[:,0]
arr = np.asarray(bench_linear_trend_FE)[:,1]
alpha_1d = np.asarray(theta_linear).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_linear_trend': arr,
    'model_linear_trend': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

(196,) (196,)
                   iso bench_time_FE  model_alpha      diff
0      X_yi_isoXtime_4      0.118579     0.007977  0.110602
1      X_yi_isoXtime_8      0.005985     0.002026  0.003959
2     X_yi_isoXtime_12      0.001192     0.001534 -0.000341
3     X_yi_isoXtime_16     -0.015025    -0.003148 -0.011877
4     X_yi_isoXtime_24      0.010997     0.001756  0.009241
..                 ...           ...          ...       ...
191  X_yi_isoXtime_704      0.004008     0.003178  0.000831
192   X_yi_isoXtime_92     -0.099697     0.005911 -0.105608
193  X_yi_isoXtime_887      0.013474     0.000843   0.01263
194  X_yi_isoXtime_894      0.000444     0.000506 -0.000061
195  X_yi_isoXtime_716     -0.001335    -0.000509 -0.000826

[196 rows x 4 columns]


In [ ]:
#time fixed effects benchmark data
bench_time=pd.read_csv('../data/Benchmark/time_fixed_effects_Burke.csv')

# cleaning up the benchmark time data
bench_time['time'] = bench_time['Unnamed: 0'].astype(str).str.extract(r'(\d{4})')
bench_time['time'] = bench_time['time'].astype(float).astype('Int64')  # nullable integer
bench_time.drop(columns=['Unnamed: 0'], inplace=True)


#comparing model time fixed effects with benchmark time fixed effects
plt.figure(figsize=(10, 6))
plt.plot(bench_time.iloc[:, 1], bench_time.iloc[:, 0], label='Benchmark', color='red')
plt.plot(model.beta, label='Model', color='blue')




## 2.3 Making 2-D plots

In [ ]:



# --- load benchmark and model predictions as before ---------------------
benchmark_data = pd.read_csv('../data/Benchmark/3d_results_Leirvik.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values


def plot_two(percentiles, fixed_value, var_name):
    fig2d = go.Figure()

    for percentile in percentiles:
        
        # flatten the precip grid and compute 33rd percentile
        fixed_array = np.percentile(np.array(fixed_value.flatten()), q=percentile)

        # find the row index in P closest to that precip
        # (assuming P is shape (n_precip, n_temp) from meshgrid)
        idx = np.argmin(np.abs(fixed_value[:,0] - fixed_array))

        # extract the corresponding temperature axis
        if var_name == 'temp':
            axis = P[idx, :]
        else: 
            axis = T[idx, :]


        # extract model and benchmark slices
        model_slice = Growth[idx, :]
        bench_slice = bench_Z[idx, :]

        # --- 2) make the 2D line plot --------------------------------------------
        fig2d.add_trace(go.Scatter(
                x=axis,
                y=model_slice,
                mode='lines',
                name=f'{var_name} {percentile}th Percentile',
                line=dict(width=2)
            ))

    
    fig2d.update_layout(
        xaxis_title='Temperature (°C)',
        yaxis_title='Growth',
        xaxis=dict(range=[0, 30]),
        legend=dict(
            bgcolor='rgba(255,255,255,0.7)',
            bordercolor='black',
            borderwidth=1
        )
    )

    fig2d.show()




percentiles= range(0, 101, 33)  # 0th, 33rd, 66th, and 100th percentiles
# --- 3) (optional) save as html ------------------------------------------

# pio.write_html(fig2d, f'../results/images/2D_slice_Leirvik_comparison_{percentile}.html', auto_open=True)


## Appendix

In [ ]:



#visualise the model: 

print(model.alpha)

In [ ]:
np=model.beta.to_numpy()

last_siz=np[:58]
print('Last 6 sizes:', last_siz)

## 2d surfaces


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px  # for a built‑in qualitative palette

# 1) compute the 90th‑percentile precip level
percentile = 50
p = np.percentile(data['PrecipPopWeight'], percentile)

# 2) tolerance window = 0.1 % of p
tol = 0.005 * p

# 3) slice all obs around that precip band
obs_slice = data[np.abs(data['PrecipPopWeight'] - p) <= tol].copy()

# 4) (no need for top‑3 any more)

# 5) base model trace
fig2d = go.Figure([
    go.Scatter(
        x=temp_axis,
        y=model_slice,
        mode='lines+markers',
        name=f'Model @ P≈{p:.1f} mm',
        line=dict(width=2, color='black'),
        marker=dict(size=6)
    )
])

# 6) pick a color palette and assign one color per country
countries = obs_slice['CountryName'].unique()
palette   = px.colors.qualitative.Plotly  # 10 distinct colors
color_map = {c: palette[i % len(palette)] for i,c in enumerate(countries)}

# 7) plot all observations, colored by country
for country in countries:
    df_ctry = obs_slice[obs_slice['CountryName'] == country]
    fig2d.add_trace(
        go.Scatter(
            x=df_ctry['TempPopWeight'],
            y=df_ctry['GrowthWDI'],
            mode='markers',
            name=country,
            marker=dict(
                size=8,
                color=color_map[country],
                symbol='circle'
            )
        )
    )

# 8) finalize layout
fig2d.update_layout(
    title=f'Growth vs Temperature at {percentile}th‑percentile Precipitation ({p:.1f} mm)',
    xaxis_title='Temperature (°C)',
    yaxis_title='Growth',
    xaxis=dict(range=[0, 30]),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig2d.show()


## Model confidence plot - based on 10 best cv models

In [ ]:

results = dict(np.load(f'../results/metrics/15042025/results.npy', allow_pickle=True).item())

# Sort the keys (node configurations) by performance metric and select the top 10.
n_models = 10
date_of_run = '15042025'
ref_model = (8,2,2)  # Reference model configuration
top_models = sorted(results, key=lambda node: results[node])[:n_models]


# Initialize a list to store the prediction surfaces for each model.
prediction_surfaces = []
time_fixed_effects=[]
country_fixed_effects=[]

for node in top_models:
    
        # Instantiate your model with the given node configuration.
    model_instance = Model(nodes=node, x_train=x_train, y_train=growth, dropout=0, formulation="global")
    weight_file = f'../results/Model Parameters/CV/{date_of_run}/{str(node)}.weights.h5'
    model_instance.load_params(weight_file)  # Load the model weights
    model_instance.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=2)
    
    model_instance.in_sample_predictions
    
    # Use the model to predict on the standardized (T,P) grid.
    growth_pred_flat = model_instance.model_visual.predict(pred_input)
    growth_pred_flat = np.reshape(growth_pred_flat, (-1,))   # shape: (900,)
    pred = growth_pred_flat.reshape(T.shape)  # reshape to (30, 30)
    
    prediction_surfaces.append(pred)
    time_fixed_effects.append(model_instance.beta)
    country_fixed_effects.append(model_instance.alpha)


# Convert the list into a NumPy array: shape (10, 30, 30)
prediction_surfaces = np.array(prediction_surfaces)

# Calculate the pointwise mean and standard deviation across the 10 models.
ensemble_mean = np.mean(prediction_surfaces, axis=0)
ensemble_std = np.std(prediction_surfaces, axis=0)

# Select the Chosen Model (e.g. configuration (8,2,2))
chosen_index = top_models.index(ref_model)
chosen_surface = prediction_surfaces[chosen_index]

model_confidence_plot(ref_model, chosen_surface, ensemble_std=ensemble_std, T=T, P=P, save_as_html=False)



## plotting the fixed effects from the model

In [ ]:

# Squeeze to shape (10, num_countries)
country_FE = np.array(country_fixed_effects).squeeze(axis=1)

# Number of models
n_models = country_FE.shape[0]

# Compute mean & sample std across the 10 models (result shape: num_countries)
country_FE_mean = np.mean(country_FE, axis=0).squeeze()
country_FE_std  = np.std(country_FE,  axis=0, ddof=1).squeeze()

# Standard error and 95% CI
se    = country_FE_std / np.sqrt(n_models)
t_val = stats.t.ppf(0.975, df=n_models - 1)
ci    = t_val * se


# X-axis indices 
country_indices = np.arange(country_FE_mean.shape[0])

plt.figure(figsize=(12, 6))
plt.plot(x, country_FE_mean, label='Mean')

plt.fill_between(
    country_indices,
    country_FE_mean - ci,
    country_FE_mean + ci,
    alpha=0.3,
    label='95% CI'
)
plt.title('Mean and 95% Confidence Interval of Country Fixed Effects')
plt.xlabel('Country Index')
plt.ylabel('Country Fixed Effect')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
